In [ ]:
# default_exp networks

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import torch
import torch.nn as nn

from torch import tensor

# Networks

> Common neural network architectures for *Collaborative Filtering*.

# Overview

This package implements several neural network architectures that can be used to build recommendation systems. Users of the library can add or define their own implementations or use the existing ones. There are two layers that every architecture should define:

* **user_embeddings**: The user embedding matrix
* **item_embeddings**: The item embedding matrix

Every implementation should be a subclass of `torch.nn.Module`.

## Simple Collaborative Filtering

This architecture is the simplest one to implement *Collaborative Filtering*. It only defines the embedding matrices for users and items and the final rating is computed by the dot product of the corresponding rows.

In [ ]:
# export
class SimpleCF(nn.Module):
    def __init__(self, n_users: int, n_items: int, factors: int = 16, 
                 user_embeddings: torch.tensor = None, freeze_users: bool = False,
                 item_embeddings: torch.tensor = None, freeze_items: bool = False,
                 init: torch.nn.init = torch.nn.init.normal_, **kwargs):
        super().__init__()
        self.user_embeddings = self._create_embedding(n_users, factors, 
                                                      user_embeddings, freeze_users, 
                                                      init, **kwargs)
        self.item_embeddings = self._create_embedding(n_items, factors, 
                                                      item_embeddings, freeze_items,
                                                      init, **kwargs)

    def forward(self, u: torch.tensor, i: torch.tensor) -> torch.tensor:
        user_embedding = self.user_embeddings(u)
        user_embedding = user_embedding[:, None, :]
        item_embedding = self.item_embeddings(i)
        item_embedding = item_embedding[:, None, :]
        rating = torch.matmul(user_embedding, item_embedding.transpose(1, 2))
        return rating
    
    def _create_embedding(self, n_items, factors, weights, freeze, init, **kwargs):
        embedding = nn.Embedding(n_items, factors)
        init(embedding.weight.data, **kwargs)
        
        if weights is not None:
            embedding.load_state_dict({'weight': weights})
        if freeze:
            embedding.weight.requires_grad = False
        
        return embedding

Arguments:

* n_users (int): The number of unique users
* n_items (int): The number of unique items
* factors (int): The dimension of the embedding space
* user_embeddings (torch.tensor): Pre-trained weights for the user embedding matrix
* freeze_users (bool): `True` if we want to keep the user weights as is (i.e. non-trainable)
* item_embeddings (torch.tensor): Pre-trained weights for the item embedding matrix
* freeze_item (bool): `True` if we want to keep the item weights as is (i.e. non-trainable)
* init (torch.nn.init): The initialization method of the embedding matrices - default: torch.nn.init.normal_

In [ ]:
# initialize the model with 100 users, 50 items and a 16-dimensional embedding space
model = SimpleCF(100, 50, 16, mean=0., std=.1)

# predict the rating that user 3 would give to item 33
model(torch.tensor([2]), torch.tensor([32]))

tensor([[-0.0205]], grad_fn=<MmBackward>)